**Imports**

In [38]:
import pandas as pd
import plotly.express as px

from os import path

**Utilities**

In [39]:
def relative(symbol: str, df: pd.DataFrame, bm: pd.DataFrame) -> pd.DataFrame:
    rel = df.reindex(bm.index, method="bfill")
    rel[symbol] = rel["close"].div(bm["close"] * rel["close"][0]) * bm["close"][0]
    return rel

def calc_relative(symbol: str, data: dict[str, pd.DataFrame]) -> pd.DataFrame:
    bm = data[symbol]
    rel = pd.DataFrame(index = bm.index)
    buffer = []
    for symbol, kline in data.items():
        try:
            df = relative(symbol, kline, bm)
            buffer.append(df[symbol])
        except TypeError:
            print(f"Error while processing {symbol}")
    rel = pd.concat(buffer, axis=1)
    return rel

**Load Data**

In [40]:
data_path = path.join("..", "data")

usdt_markets = pd.read_csv(path.join(data_path, "USDT_markets.csv"))
btc_markets = pd.read_csv(path.join(data_path, "BTC_markets.csv"))

usdt_minutly = {}
usdt_hourly = {}
usdt_daily = {}

for market in usdt_markets["symbol"]:
    usdt_minutly[market] = pd.read_csv(path.join(data_path, f"{market}_minutly.csv"), index_col="time")
    usdt_hourly[market] = pd.read_csv(path.join(data_path, f"{market}_hourly.csv"), index_col="time")
    usdt_daily[market] = pd.read_csv(path.join(data_path, f"{market}_daily.csv"), index_col="time")

**Calculate Relative Performance**

In [41]:
intra = {k: v[-96:] for k, v in usdt_minutly.items()}
day = {k: v[-24:] for k, v in usdt_hourly.items()}
week = {k: v[(-24*7):] for k, v in usdt_hourly.items()}
month = {k: v[-30:] for k, v in usdt_daily.items()}

rel_intra = calc_relative("BTCUSDT", intra)
rel_day = calc_relative("BTCUSDT", day)
rel_week = calc_relative("BTCUSDT", week)
rel_month = calc_relative("BTCUSDT", month)
rel_year = calc_relative("BTCUSDT", usdt_daily)

diff = (rel_intra < 1.05) & (rel_intra > 0.95)
rel_intra = rel_intra.drop(columns=rel_intra.columns[diff.all()])

diff = (rel_day < 1.10) & (rel_day > 0.90)
rel_day = rel_day.drop(columns=rel_day.columns[diff.all()])

diff = (rel_week < 1.15) & (rel_week > 0.85)
rel_week = rel_week.drop(columns=rel_week.columns[diff.all()])

diff = (rel_month < 1.3) & (rel_month > 0.7)
rel_month = rel_month.drop(columns=rel_month.columns[diff.all()])

diff = (rel_year < 4.0) & (rel_year > 0.25)
rel_year = rel_year.drop(columns=rel_year.columns[diff.all()])

**Plotting**

In [42]:
fig_i = px.line(rel_intra, x = rel_intra.index, y = rel_intra.columns)
fig_d = px.line(rel_day, x = rel_day.index, y = rel_day.columns)
fig_w = px.line(rel_week, x = rel_week.index, y = rel_week.columns)
fig_m = px.line(rel_month, x = rel_month.index, y = rel_month.columns)
fig_y = px.line(rel_year, x = rel_year.index, y = rel_year.columns)

fig_i.show()
fig_d.show()
fig_w.show()
fig_m.show()
fig_y.show()